In [ ]:
from moviepy.editor import VideoFileClip
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [4]:
# Calibrating camera

obj_points = []
img_points = []

nx = 9
ny = 6

cal_paths = glob.glob('./camera_cal/*.jpg')

objp = np.zeros((nx*ny,3),np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

for i, path in enumerate(cal_paths):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray,(nx,ny))
    if ret == True:
        obj_points.append(objp)
        img_points.append(corners)
        img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret) 
        plt.figure(i)
        plt.imshow(img)
        
retval, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, 
                                                                     gray.shape, None, None) 
print (cameraMatrix)

[[  1.15396100e+03   0.00000000e+00   6.69706490e+02]
 [  0.00000000e+00   1.14802504e+03   3.85655584e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]


In [5]:
from moviepy.editor import VideoFileClip
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def nothing(img):
    pass


#cv2.startWindowThread()
cv2.namedWindow('image')

test_paths = glob.glob('./test_images/*.jpg')
#img = np.zeros((300,512,3), np.uint8)
cv2.createTrackbar('Sobel kernel','image',1,3,nothing)
cv2.createTrackbar('Image number','image',0,len(test_paths),nothing)


while(1):
    # get current positions of four trackbars
    kernel_size = 1 + 2*cv2.getTrackbarPos('Sobel kernel','image')
    img_id = cv2.getTrackbarPos('Image number','image')

    img = mpimg.imread(test_paths[img_id])
    #cv2.imshow('image',img)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
    #cv2.imshow('image',img)
    img_undist = cv2.undistort(img, cameraMatrix, distCoeffs)
    img_hls = cv2.cvtColor(img_undist,cv2.COLOR_RGB2HLS)
    img_sat = img_hls[:,:,2]
    sobel_x = cv2.Sobel(img_sat,cv2.CV_64F,1,0,ksize=kernel_size)
    sobel_y = cv2.Sobel(img_sat,cv2.CV_64F,0,1,ksize=kernel_size)
    sobel_x = np.absolute(sobel_x)
    sobel_y = np.absolute(sobel_y)
    grad = np.sqrt(np.power(sobel_x,2)+np.power(sobel_y,2))
    img_bin = np.asarray(255*grad/np.max(grad),dtype=np.uint8)
    cv2.imshow('image',img_bin)

IndexError: list index out of range

In [ ]:
from moviepy.editor import VideoFileClip
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def nothing(x):
    pass

# Create a black image, a window
img = np.zeros((300,512,3), np.uint8)
cv2.namedWindow('image')

# create trackbars for color change
cv2.createTrackbar('R','image',0,255,nothing)
cv2.createTrackbar('G','image',0,255,nothing)
cv2.createTrackbar('B','image',0,255,nothing)

# create switch for ON/OFF functionality
switch = '0 : OFF \n1 : ON'
cv2.createTrackbar(switch, 'image',0,1,nothing)

while(1):
    cv2.imshow('image',img)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

    # get current positions of four trackbars
    r = cv2.getTrackbarPos('R','image')
    g = cv2.getTrackbarPos('G','image')
    b = cv2.getTrackbarPos('B','image')
    s = cv2.getTrackbarPos(switch,'image')

    if s == 0:
        img[:] = 0
    else:
        img[:] = [b,g,r]

cv2.destroyAllWindows()

In [ ]:
test_paths = glob.glob('./test_images/*.jpg')
cv2.startWindowThread()
cv2.namedWindow('image')

cv2.createTrackbar('Sobel kernel','image',0,3,nothing)

for i, path in enumerate(test_paths):
    img = mpimg.imread(path)
    cv2.imshow('image',img)
    img_undist = cv2.undistort(img, cameraMatrix, distCoeffs)
    
    img_hls = cv2.cvtColor(img_undist,cv2.COLOR_RGB2HLS)
    img_sat = img_hls[:,:,2]
    sobel_x = cv2.Sobel(img_sat,cv2.CV_64F,1,0,ksize=5)
    sobel_y = cv2.Sobel(img_sat,cv2.CV_64F,0,1,ksize=5)
    sobel_x = np.absolute(sobel_x)
    sobel_y = np.absolute(sobel_y)
    grad = np.sqrt(np.power(sobel_x,2)+np.power(sobel_y,2))
    img_bin = np.asarray(255*grad/np.max(grad),dtype=np.uint8)
    
    plt.figure(i,figsize=(20,10))
    plt.subplot(121)
    plt.imshow(img_undist)
    plt.subplot(122)
    plt.imshow(img_bin, cmap='gray')

In [ ]:
test_paths = glob.glob('./test_images/*.jpg')

for i, path in enumerate(test_paths):
    img = mpimg.imread(path)
    img_undist = cv2.undistort(img, cameraMatrix, distCoeffs)
    #plt.figure(i,figsize=(20,10))
    #plt.subplot(121)
    #plt.imshow(img)
    #plt.subplot(122)
    #plt.imshow(img_undist)

    img_draw = img.copy()
    src = np.int32([[580,458],[700,458],[1032,671],[274,671]])
    dst = np.int32([[274+50,0],[1032-50,0],[1032-50,img.shape[1]],[274+50,img.shape[1]]])
    cv2.polylines(img_draw, [src], True, (0,255,0),5)
    cv2.polylines(img_draw, [dst], True, (0,0,255),5)

    M = cv2.getPerspectiveTransform(np.asarray(src,dtype=np.float32),np.asarray(dst,dtype=np.float32))
    img_warp = cv2.warpPerspective(img_undist, M, (img.shape[1],img.shape[0]))

    plt.figure(i,figsize=(20,10))
    plt.subplot(121)
    plt.imshow(img_draw)
    plt.subplot(122)
    plt.imshow(img_warp)